In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_with_lag/data_with_lag.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_1"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold_current  Temp_Very_Cold_prev  \
0     148.828353                       0                  0.0   
1     159.793757                       0                  0.0   
2     111.885594                       0                  0.0   
3     168.864941                       0                  0.0   
4     171.280754                       0                  0.0   
...          ...                     ...                  ...   
1814  198.225523                       0                  0.0   
1815  214.470393                       0                  0.0   
1816  129.801991                       0                  0.0   
1817  148.541340                       0                  0.0   
1818  123.075682                       0                  0.0   

      Temp_Very_Cold_next  Temp_Cold_current  Temp_Cold_prev  Temp_Cold_next  \
0                     0.0                  0             0.0             0.0   
1                     0.0                  0             0.

In [8]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Zeilen- und Spaltenanzeige in Pandas erweitern
pd.set_option("display.max_rows", None)  # Zeigt alle Zeilen
pd.set_option("display.max_columns", None)  # Zeigt alle Spalten

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                            1.000000
Ostertag_current                  0.343797
Werktag_prev                      0.267577
is_holiday_current                0.261852
Samstag_current                   0.261361
Sommer_prev                       0.246935
Sonntag_next                      0.246710
Sommer_current                    0.244808
Sommer_next                       0.241408
is_holiday_prev                   0.233458
Freitag_prev                      0.231008
is_holiday_next                   0.230539
Werktag_current                   0.188045
Monat_8_next                      0.177330
Monat_8_current                   0.174328
Monat_8_prev                      0.165440
Ostertag_next                     0.162691
Monat_7_prev                      0.150243
Silvester_current                 0.144596
Mittwoch_prev                     0.144151
Monat_7_next                      0.138403
Monat_7_current                   0.136877
Feiertag_next                     0.136712
Donnerstag_

In [3]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


Sonntag_current     0.176498
Samstag_prev        0.173074
Montag_next         0.168519
Werktag_current     0.089884
VPI_next            0.086701
                      ...   
Monat_10_current    0.000000
Monat_10_prev       0.000000
Wind_ok_current     0.000000
Wind_ok_prev        0.000000
Wind_ok_next        0.000000
Length: 168, dtype: float64


In [4]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Temp_Cold_current', 'Temp_Cold_prev', 'Temp_Cold_next', 'Cloud_Cloudy_current', 'Cloud_Cloudy_prev', 'Cloud_Cloudy_next', 'Wind_Light_current', 'Wind_Light_prev', 'Wind_Light_next', 'Wind_Moderate_current', 'Wind_Moderate_prev', 'Wind_Moderate_next', 'Wind_Strong_current', 'Wind_Strong_prev', 'Wind_Strong_next', 'KielerWoche_current', 'KielerWoche_prev', 'KielerWoche_next', 'Montag_current', 'Montag_prev', 'Montag_next', 'Dienstag_current', 'Dienstag_prev', 'Dienstag_next', 'Mittwoch_current', 'Mittwoch_prev', 'Mittwoch_next', 'Donnerstag_current', 'Donnerstag_prev', 'Donnerstag_next', 'Freitag_current', 'Freitag_prev', 'Freitag_next', 'Samstag_current', 'Samstag_prev', 'Samstag_next', 'Sonntag_current', 'Sonntag_prev', 'Sonntag_next', 'VPI_current', 'VPI_prev', 'VPI_next', 'Number_of_ships_current', 'Number_of_ships_prev', 'Number_of_ships_next', 'Ship_current', 'Ship_prev', 'Ship_next', 'is_holiday_current', 'is_holiday_prev', 'is_holiday_next', 'Weihnac

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                    Feature           VIF
0    Temp_Very_Cold_current  2.942845e+08
1       Temp_Very_Cold_prev           inf
2       Temp_Very_Cold_next           inf
3         Temp_Cold_current           inf
4            Temp_Cold_prev  2.404989e+07
..                      ...           ...
163           Cloud_ok_prev           inf
164           Cloud_ok_next           inf
165         Wind_ok_current           inf
166            Wind_ok_prev           inf
167            Wind_ok_next           inf

[168 rows x 2 columns]


In [6]:
# suche miteinander korrelierrende features

In [7]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Temp_Mild_prev', 'Temp_Cold_prev'), ('Monat_5_next', 'Monat_5_current'), ('Freitag_current', 'Donnerstag_prev'), ('Monat_7_current', 'Sommer_current'), ('Monat_7_current', 'Sommer_next'), ('Monat_10_current', 'Herbst_next'), ('Temp_Mild_next', 'Temp_Cold_prev'), ('Monat_4_next', 'Markt_next'), ('Temp_Mild_next', 'Temp_Cold_current'), ('Sommer_current', 'Temp_Cold_prev'), ('Sommer_current', 'Temp_Cold_current'), ('Monat_2_next', 'Winter_next'), ('Heimspiel_current', 'Samstag_current'), ('Sonntag_current', 'Montag_next'), ('Monat_3_next', 'Monat_3_current'), ('Monat_7_prev', 'Sommer_prev'), ('Monat_7_next', 'Monat_7_prev'), ('zwischen_den_jahren_next', 'Silvester_next'), ('Frühling_prev', 'Frühling_current'), ('zwischen_den_jahren_prev', 'Silvester_prev'), ('Monat_6_current', 'KielerWoche_prev'), ('Monat_2_prev', 'Winter_next'), ('Monat_4_next', 'Frühling_current'), ('Monat_6_prev', 'Sommer_current'), ('Monat_8_current', 'Sommer_current'), ('Monat_8_cu